In [1]:
#export
from exp.nb_05b import *
torch.set_num_threads(2)

### ConvNet

In [2]:
x_train,y_train,x_valid,y_valid = get_data()

In [3]:
#export
def normalize_to(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [4]:
x_train,x_valid = normalize_to(x_train,x_valid)
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

In [5]:
x_train.mean(),x_train.std()

(tensor(0.0001), tensor(1.))

In [6]:
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [7]:
#export
class Lambda(nn.Module):
    def __init__(self,func):
        super().__init__()
        self.func = func
    def forward(self,x): return self.func(x)
    
def flatten(x): return x.view(x.shape[0],-1)

In [8]:
def mnist_resize(x): return x.view(-1,1,28,28)

In [9]:
def get_cnn_model(data):
    return nn.Sequential(
        Lambda(mnist_resize),
        nn.Conv2d(1,8,5,padding=2,stride=2),nn.ReLU(), #14
        nn.Conv2d(8,16,3,padding=1,stride=2),nn.ReLU(), #7
        nn.Conv2d(16,32,3,padding=1,stride=2),nn.ReLU(),#4
        nn.Conv2d(32,32,3,padding=1,stride=2),nn.ReLU(),#2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,data.c)
    )

In [10]:
model = get_cnn_model(data)

In [14]:
cbfs=[Recorder,partial(AvgStatsCallback,accuracy)]

In [17]:
opt=optim.SGD(model.parameters(),lr=0.4)
learn=Learner(model,opt,loss_func,data)
run=Runner(cb_funcs=cbfs)

In [18]:
%time run.fit(1,learn)

train: [1.7253144254389496, tensor(0.4136)]
valid: [0.5480818359375, tensor(0.8332)]
CPU times: user 3.57 s, sys: 668 ms, total: 4.23 s
Wall time: 4.26 s


### CUDA

In [19]:
#more flexible way
device = torch.device('cuda',0)

In [20]:
class CudaCallback(Callback):
    def __init__(self,device): self.device=device
    def begin_fit(self): self.model.to(device)
    def begin_batch(self):self.run.xb,self.run.yb=self.xb.to(device),self.yb.to(device)

In [ ]:
#less flexible but quite convenient
torch.cuda.set_device(device)

In [25]:
if not hasattr(torch._C, '_cuda_setDevice'):
    print('CUDA not available. Setting device=-1.')
    cuda_id = -1
    torch.cuda.set_device(cuda_id)
else:
    print('Setting CUDA.')
    torch.cuda.set_device(device)

CUDA not available. Setting device=-1.


In [26]:
#export
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [27]:
cbfs.append(CudaCallback)

In [28]:
model = get_cnn_model(data)

In [29]:
opt = optim.SGD(model.parameters(),lr=0.4)
learn = Learner(model,opt,loss_func,data)
run=Runner(cb_funcs=cbfs)

In [ ]:
%time run.fit(3,learn)